In [194]:
import os
import json
from typing import Optional, List
from types import NoneType

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages.ai import AIMessage
from IPython.display import Markdown, display
from pydantic import BaseModel, Field
from langchain.tools import tool
import subprocess
from pathlib import Path

from sigmaepsilon.math.function import Function, Relation
from sigmaepsilon.math.optimize import LinearProgrammingProblem as LPP
import sympy as sy

In [195]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
model = ChatOpenAI(model=MODEL, temperature=0)

OpenAI API Key exists and begins sk-proj-


In [196]:
# You can use llm.invoke with a list of messages (dictionaries with "role" and "content")
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me a joke!"}
]
response = model.invoke(messages)
response.content

"Sure! Here's a joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"

In [197]:
system_message = """
You are a helpful assistant who solves linear programming problems.
You will receive the business definition of a linear programming problem.
Your task is to formulate the mathematical model, including the objective function and constraints.
Then, you will solve the linear programming problem using the provided optimization tool.
Provide the optimal solution and a brief explanation of the results.

Rules:
    - Always be accurate. If you don't know the answer, say so.
    - Always formulate the mathematical optimization problem as a minimization problem.
    - Use the optimization tool to solve the linear programming problem.
    - Recognize from the context if variables should be continuous or integer valued and provide this information to the optimization tool.
"""

In [198]:



class OptimizationProblemInput(BaseModel):
    """Input for a linear programming problem."""
    objective: str = Field(
        description="The objective function to be minimized as a string, e.g., '3*x + 4*y'."
    )
    constraints: List[str] = Field(
        description="List of constraints as strings, e.g., ['x + 2*y <= 10', 'x >= 0', 'y >= 0']."
    )
    variables: List[str] = Field(
        description="List of variable names, e.g., ['x', 'y']."
    )
    bounds: Optional[List[List[Optional[float]]]] = Field(
        default=None,
        description="Optional list of bounds for each variable as [lower, upper], e.g., [[0, None], [0, None]]. Use null for unbounded."
    )
    integrality: Optional[List[int]] = Field(
        default=None,
        description="A 0 or a 1 for each variable indicating if it should be integer (1) or continuous (0), e.g., [0, 1]."
    )


@tool(args_schema=OptimizationProblemInput)
def solve_linear_programming_problem(
        objective: str, 
        constraints: list[str], 
        variables: list[str], 
        bounds: Optional[list[tuple[float | NoneType, float | NoneType]]] = None,
        integrality: Optional[list[int]] = None
    ) -> str:
    """Solves a linear programming problem given an objective function, constraints, variables, bounds, and integrality conditions.
    
    Parameters
    ----------
    objective : str
        The objective function to be minimized as a string, e.g., '3x + 4y'.
    constrains : list[str]
        List of constraints as strings, e.g., ['x + 2y <= 10', 'x >= 0', 'y >= 0'].
    variables : list[str]
        List of variable names, e.g., ['x', 'y'].
    bounds : Optional[list[tuple[float | NoneType, float | NoneType]]], optional
        Optional list of bounds for each variable as tuples, e.g., [(0, None), (0, None)], by default None.
    integrality : Optional[list[int]], optional
        A 0 or a 1 for each variable indicating if it should be integer (1) or continuous (0), e.g., [0, 1], by default None.
        
    Returns
    -------
    str
        A JSON string containing the optimal solution, objective function value, status, message, success flag, and number of iterations.
    """
    print("Solving LPP with:")
    print(f"Objective: {objective}")
    print(f"Constraints: {constraints}")
    print(f"Variables: {variables}")
    print(f"Bounds: {bounds}")
    print(f"Integrality: {integrality}")
    vars = sy.symbols(variables)
    obj = Function(objective, variables=vars)
    relations = [Relation(expression, variables=vars) for expression in constraints]
    lpp = LPP(obj, relations, variables=vars, bounds=bounds, integrality=integrality)
    solution = lpp.solve()
    
    result = {
        "solution": solution.x.tolist(),
        "objective_function_value": solution.fun,
        "status": solution.status,
        "message": solution.message,
        "success": solution.success,
        "number_of_iterations": solution.nit,
    }
    return json.dumps(result)


tools = [solve_linear_programming_problem]


solve_linear_programming_problem.func(
    objective="3*X1 + 5*X2",
    constraints=["2*X1 + 3*X2 <= 12", "X1 + X2 <= 5"],
    variables=["X1", "X2"],
    bounds=[(0, None), (0, None)]
)

Solving LPP with:
Objective: 3*X1 + 5*X2
Constraints: ['2*X1 + 3*X2 <= 12', 'X1 + X2 <= 5']
Variables: ['X1', 'X2']
Bounds: [(0, None), (0, None)]
Integrality: None


'{"solution": [0.0, 0.0], "objective_function_value": 0.0, "status": 0, "message": "Optimization terminated successfully. (HiGHS Status 7: Optimal)", "success": true, "number_of_iterations": 0}'

In [199]:
model_with_tools = model.bind_tools(tools)

In [200]:
with open("problem_1.md", "r", encoding="utf-8") as f:
    user_message = f.read()

In [201]:
display(Markdown(user_message))

# **Problem Statement**

A small manufacturing company produces two types of office chairs: **Standard** and **Ergonomic**. The company aims to determine how many units of each chair to produce next month to **maximize profit**, while working within production capacity limits and market constraints.

## **Product Profit**

- Each **Standard chair** yields a profit of **€30**.
- Each **Ergonomic chair** yields a profit of **€50**.

## **Production Requirements and Capacity**

Production involves two departments:

### **Carpentry Department**

- Time needed per Standard chair: **2 hours**
- Time needed per Ergonomic chair: **3 hours**
- Total carpentry hours available next month: **600 hours**

### **Upholstery Department**

- Time needed per Standard chair: **1 hour**
- Time needed per Ergonomic chair: **2 hours**
- Total upholstery hours available next month: **400 hours**

## **Market and Policy Constraints**

- Due to demand limits, the company can sell **at most 150 Ergonomic chairs** next month.
- Because of a long-term supply agreement, the company must produce **at least 100 Standard chairs**.
- Naturally, production quantities cannot be negative.


In [202]:
# You can use llm.invoke with a list of messages (dictionaries with "role" and "content")
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]
response = model_with_tools.invoke(messages)
display(Markdown(response.content))

Let's define the decision variables:

- \( x \): Number of Standard chairs to produce
- \( y \): Number of Ergonomic chairs to produce

### Objective Function
We want to maximize profit:
\[
\text{Maximize } Z = 30x + 50y
\]

Since we need to formulate this as a minimization problem, we minimize the negative profit:
\[
\text{Minimize } -Z = -30x - 50y
\]

### Constraints

1. Carpentry department hours:
\[
2x + 3y \leq 600
\]

2. Upholstery department hours:
\[
x + 2y \leq 400
\]

3. Demand limit for Ergonomic chairs:
\[
y \leq 150
\]

4. Minimum production for Standard chairs:
\[
x \geq 100
\]

5. Non-negativity:
\[
x \geq 0, \quad y \geq 0
\]

Since \( x \) and \( y \) represent number of chairs, they should be integer values.

---

### Mathematical Model

\[
\begin{align*}
\text{Minimize } & -30x - 50y \\
\text{subject to } & \\
& 2x + 3y \leq 600 \\
& x + 2y \leq 400 \\
& y \leq 150 \\
& x \geq 100 \\
& x, y \geq 0 \\
& x, y \in \mathbb{Z}
\end{align*}
\]

I will now solve this integer linear programming problem.

In [203]:
response

AIMessage(content="Let's define the decision variables:\n\n- \\( x \\): Number of Standard chairs to produce\n- \\( y \\): Number of Ergonomic chairs to produce\n\n### Objective Function\nWe want to maximize profit:\n\\[\n\\text{Maximize } Z = 30x + 50y\n\\]\n\nSince we need to formulate this as a minimization problem, we minimize the negative profit:\n\\[\n\\text{Minimize } -Z = -30x - 50y\n\\]\n\n### Constraints\n\n1. Carpentry department hours:\n\\[\n2x + 3y \\leq 600\n\\]\n\n2. Upholstery department hours:\n\\[\nx + 2y \\leq 400\n\\]\n\n3. Demand limit for Ergonomic chairs:\n\\[\ny \\leq 150\n\\]\n\n4. Minimum production for Standard chairs:\n\\[\nx \\geq 100\n\\]\n\n5. Non-negativity:\n\\[\nx \\geq 0, \\quad y \\geq 0\n\\]\n\nSince \\( x \\) and \\( y \\) represent number of chairs, they should be integer values.\n\n---\n\n### Mathematical Model\n\n\\[\n\\begin{align*}\n\\text{Minimize } & -30x - 50y \\\\\n\\text{subject to } & \\\\\n& 2x + 3y \\leq 600 \\\\\n& x + 2y \\leq 400 \\

In [204]:
response.tool_calls

[{'name': 'solve_linear_programming_problem',
  'args': {'objective': '-30*x - 50*y',
   'constraints': ['2*x + 3*y <= 600',
    'x + 2*y <= 400',
    'y <= 150',
    'x >= 100',
    'x >= 0',
    'y >= 0'],
   'variables': ['x', 'y'],
   'bounds': [[100, None], [0, 150]],
   'integrality': [1, 1]},
  'id': 'call_AbZaUk4NLZtjnwXEaJR4T8zI',
  'type': 'tool_call'}]

In [205]:
def handle_tool_calls(model_response: AIMessage):
    responses = []
    for tool_call in model_response.tool_calls:
        if tool_call["name"]== "solve_linear_programming_problem":
            tool_result = solve_linear_programming_problem.invoke(tool_call)
            responses.append(tool_result)
    return responses


history = []


def chat(message: list[str]) -> str:
    global history
    
    messages = (
        [{"role": "system", "content": system_message}] 
        + history 
        + [{"role": "user", "content": message}]
    )
    response = model_with_tools.invoke(messages)
    messages.append(response)
    history.extend(messages)
    
    while len(response.tool_calls) > 0:
        tool_responses = handle_tool_calls(response)
        messages.extend(tool_responses)
        history.extend(tool_responses)
        response = model_with_tools.invoke(messages)
    
    return response.content

In [206]:
content = chat(user_message)

Solving LPP with:
Objective: -30*x - 50*y
Constraints: ['2*x + 3*y <= 600', 'x + 2*y <= 400', 'y <= 150', 'x >= 100', 'x >= 0', 'y >= 0']
Variables: ['x', 'y']
Bounds: [[100.0, None], [0.0, 150.0]]
Integrality: [1, 1]


In [207]:
display(Markdown(content))

The optimal solution is to produce:

- 102 Standard chairs
- 132 Ergonomic chairs

This production plan will maximize the profit, yielding a total profit of €9660.

Explanation:
- The solution respects all constraints: carpentry and upholstery hours, demand limit for ergonomic chairs, and minimum production for standard chairs.
- The profit is maximized by producing close to the maximum allowed ergonomic chairs (132 out of 150) and just above the minimum required standard chairs (102).

In [208]:
content = chat((
    "Now give me the full LaTeX code of a summary about the problem and its solution. "
    "Respond with the LaTeX code only."
))

In [209]:
content

'```latex\n\\documentclass{article}\n\\usepackage{amsmath}\n\\usepackage{amssymb}\n\\usepackage{geometry}\n\\geometry{a4paper, margin=1in}\n\n\\title{Optimization of Chair Production for Maximum Profit}\n\\author{Manufacturing Company}\n\\date{\\today}\n\n\\begin{document}\n\n\\maketitle\n\n\\section*{Problem Summary}\n\nA small manufacturing company produces two types of office chairs: \\textbf{Standard} and \\textbf{Ergonomic}. The goal is to determine the optimal number of units of each chair to produce next month in order to maximize profit, while respecting production capacity and market constraints.\n\n\\subsection*{Profit per Unit}\n\\begin{itemize}\n    \\item Standard chair: €30\n    \\item Ergonomic chair: €50\n\\end{itemize}\n\n\\subsection*{Production Capacity}\n\\begin{itemize}\n    \\item \\textbf{Carpentry Department:}\n    \\begin{itemize}\n        \\item Time per Standard chair: 2 hours\n        \\item Time per Ergonomic chair: 3 hours\n        \\item Total available h

In [211]:
def clean_content(content:str) -> str:
    raw = content.strip()

    # Remove starting ```latex
    if raw.startswith("```latex"):
        raw = raw[len("```latex"):].strip()

    # Remove ending ```
    if raw.endswith("```"):
        raw = raw[:-3].strip()
    
    return raw


def save_latex_to_file(content: str, tex_path: str = "problem_1.tex") -> Path:
    cleaned_content = clean_content(content)
    tex_file = Path(tex_path).resolve()
    tex_file.write_text(cleaned_content, encoding="utf-8")
    return tex_file


def compile_latex(tex_file: Path, engine: str = "pdflatex") -> Path:
    """
    Compile a .tex file to PDF using the given LaTeX engine (pdflatex, xelatex, etc.).
    Returns the Path to the generated PDF.
    """
    # Run in the .tex file's directory
    workdir = tex_file.parent
    tex_name = tex_file.name

    # First compilation
    result = subprocess.run(
        [engine, "-interaction=nonstopmode", tex_name],
        cwd=workdir,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
    )

    if result.returncode != 0:
        print("LaTeX compilation failed.")
        print("STDOUT:\n", result.stdout)
        print("STDERR:\n", result.stderr)
        raise RuntimeError(f"{engine} exited with code {result.returncode}")

    # Optional: run a second time for references/TOC
    subprocess.run(
        [engine, "-interaction=nonstopmode", tex_name],
        cwd=workdir,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        text=True,
    )

    pdf_file = tex_file.with_suffix(".pdf")
    if not pdf_file.exists():
        raise FileNotFoundError(f"Expected PDF not found: {pdf_file}")

    return pdf_file


tex_path = save_latex_to_file(content)
pdf_path = compile_latex(tex_path)

print(f"PDF generated at: {pdf_path}")

PDF generated at: /Users/baloghbence/Documents/Projects/sigmaepsilon.math/apps/llm_optimizer/problem_1.pdf
